In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/wm-project/"
%cd /content/drive/My\ Drive/wm-project/

Mounted at /content/drive
 02062021MusicRecommenderTeam3.ipynb
 26052021MusicRecommenderTeam3.ipynb
 all_predictions_scores.csv
 als_predictions_scores1.csv
 als_predictions_scores2.csv
 als_predictions_scores.csv
 back22052021MusicRecommenderTeam3.ipynb
 df_artist.csv
 df_general.csv
 df_outlierDtected.csv
 df_preprocessed.csv
 df_test.csv
 df_train_cleaned.csv
 df_train_to_explore.csv
 lookup_artist_names.csv
 MusicRecommender_01_Preprocessing_backup.ipynb
 MusicRecommender_01_Preprocessing.ipynb
 MusicRecommender_02_Modelling.ipynb
 nrartist_df.csv
 parameter_tuning
 rec_train.csv
 tasks.gsheet
'Team 3 - Music recommender.gslides'
 userid-profile.tsv
 userid-timestamp-artid-artname-traid-traname.tsv
 WM_Slides_08062021.pptx
/content/drive/My Drive/wm-project


In [2]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 34.5MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617622 sha256=5deee3beecf6895abd6625546e0b32c5b860bb15327b6042106944b8a6c79177
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import hashlib
import pytz
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.sparse import csr_matrix
from surprise import SVD, NMF, KNNBasic, KNNWithMeans,Reader,Dataset
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import ndcg_score
from scipy.sparse import csr_matrix
from sklearn.preprocessing import minmax_scale

In [ ]:
#df = pd.read_csv('df_train_to_explore.csv')

In [17]:
trainset = pd.read_csv('df_train_cleaned.csv')
testset = pd.read_csv('df_test.csv')

In [18]:
trainset = trainset.drop_duplicates() # 992 users
testset = testset.drop_duplicates() # 990 users

In [22]:
trainset_ = trainset.drop(columns=['absolute', 'user_total'])

testset_= testset.copy()
testset_['user_total'] = testset_.groupby('user_id').absolute.transform('sum')

testset_['relative'] = testset_['absolute'] / testset_['user_total'] 

testset_.drop(columns=['user_total','absolute'], inplace=True)
testset_.head(4)

,user_id,artist_id,relative
0,user_000766,95e1ead9-4d31-4808-a7ac-32c3614c116b,0.010989
1,user_000920,eace2373-31c8-4aba-9a5c-7bce22dd140a,0.013481
2,user_000591,eeb1195b-f213-4ce1-b28c-8565211f8e43,0.001837
3,user_000755,2a3eb033-5077-4fc7-bc49-c15dc6b3a691,0.000190


In [23]:
trainset_.describe()

,relative
count,216822.000000
mean,0.004571
std,0.013049
min,0.000023
25%,0.000416
50%,0.001222
75%,0.004084
max,1.000000


In [12]:
from surprise import KNNBasic,Reader,KNNWithMeans,KNNWithZScore,Dataset
from surprise.model_selection import cross_validate,GridSearchCV


#reader = Reader(line_format= 'user item rating',rating_scale=(0, 100))
reader = Reader(line_format= 'user item rating',rating_scale=(0, 1))

trainset_mf = Dataset.load_from_df(trainset_, reader)


In [13]:


sim_option_item = {'name': 'cosine',
              'user_based': False  # compute similarities between items
               }
sim_option_user = {'name': 'pearson',
              'user_based': True  # compute similarities between users
               }
grid_search = {
    'sim_option_item': [sim_option_item,sim_option_user],
    'k': [60],
    # 5,10,20,40,
    'min_1': [1]}
              #,2,3,4,5]

grid_search = GridSearchCV(KNNWithMeans , param_grid = grid_search, 
                              measures=['RMSE','MAE'], cv = 5, n_jobs = -1)

grid_search.fit(trainset_mf)


In [15]:
print('best configuration based on RMSE : ')
print(grid_search.best_params)



best configuration based on RMSE : 
{'rmse': {'sim_option_item': {'name': 'cosine', 'user_based': False}, 'k': 60, 'min_1': 1}, 'mae': {'sim_option_item': {'name': 'cosine', 'user_based': False}, 'k': 60, 'min_1': 1}}


In [16]:
print(grid_search.best_score)

{'rmse': 0.012400023338853549, 'mae': 0.004596162769921594}


**The result of gridsearch for relative (0,1) :**  \\

best configuration based on RMSE : 
{'rmse': {'sim_option_item': {'name': 'cosine', 'user_based': False}, 'k': 60, 'min_1': 1}, 'mae': {'sim_option_item': {'name': 'cosine', 'user_based': False}, 'k': 60, 'min_1': 1}}

**The result of gridsearch for relative (0,100) :** \\

best configuration based on RMSE : 
{'rmse': {'sim_option_item': {'name': 'cosine', 'user_based': False}, 'k': 60, 'min_1': 1}, 'mae': {'sim_option_item': {'name': 'cosine', 'user_based': False}, 'k': 60, 'min_1': 1}}